In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [3]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', None)        
pd.set_option('display.expand_frame_repr', False)

In [4]:
X_train = df.drop(columns=['SalePrice'])
y_train = df['SalePrice']

In [5]:
X_train.drop('Id', axis=1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,NaN,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,1,240,TA,TA,Y,366,0,112,0,0,0,NaN,NaN,Na

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class HousePricePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, meaningful_na_columns, categorical_cols, important_numerical_cols, 
                 ordinal_mappings, nominal_cols, null_threshold=0.8, max_onehot=4):
        self.meaningful_na_columns = meaningful_na_columns
        self.categorical_cols = categorical_cols  
        self.important_numerical_cols = important_numerical_cols
        self.ordinal_mappings = ordinal_mappings
        self.nominal_cols = nominal_cols  
        self.null_threshold = null_threshold  
        self.max_onehot = max_onehot  
        self.categorical_modes = None
        self.numerical_medians = None
        self.columns_to_drop = None
        self.valid_onehot_cols = None  
        self.high_card_cols = None  
        
    def fit(self, X, y=None):
        null_ratios = X.isnull().mean()
        self.columns_to_drop = list(null_ratios[null_ratios > self.null_threshold].index)
        
        remaining_categorical = [col for col in self.categorical_cols if col not in self.columns_to_drop]
        self.categorical_modes = X[remaining_categorical].mode().iloc[0].to_dict()
        
        remaining_numerical = [col for col in self.important_numerical_cols if col not in self.columns_to_drop]
        self.numerical_medians = X[remaining_numerical].median().to_dict()
        
        remaining_nominal = [col for col in self.nominal_cols 
                           if col not in self.columns_to_drop and col in X.columns]
        
        self.valid_onehot_cols = []
        self.high_card_cols = []
        
        for col in remaining_nominal:
            unique_count = X[col].nunique()
            if unique_count <= self.max_onehot:
                self.valid_onehot_cols.append(col)
            else:
                self.high_card_cols.append(col)
        self.feature_columns_ = None
        
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        added_columns = []
        X_transformed = X_transformed.drop(columns=self.columns_to_drop, errors='ignore')
        
        meaningful_na_columns = [col for col in self.meaningful_na_columns if col not in self.columns_to_drop]
        categorical_cols = [col for col in self.categorical_cols if col not in self.columns_to_drop]
        important_numerical_cols = [col for col in self.important_numerical_cols if col not in self.columns_to_drop]
        
        X_transformed[meaningful_na_columns] = X_transformed[meaningful_na_columns].fillna("None")
        
        for col in categorical_cols:
            if col not in meaningful_na_columns and col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.categorical_modes.get(col, "None"))
        
        for col in important_numerical_cols:
            if col in X_transformed.columns:
                X_transformed[col] = X_transformed[col].fillna(self.numerical_medians[col])
        
        if all(col in X_transformed.columns for col in ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']):
            X_transformed['TotalSF'] = X_transformed['TotalBsmtSF'] + X_transformed['1stFlrSF'] + X_transformed['2ndFlrSF']
        added_columns.append('TotalSF')
        
        porch_cols = ['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        if all(col in X_transformed.columns for col in porch_cols):
            X_transformed['TotalPorchSF'] = (
                X_transformed['OpenPorchSF'] + 
                X_transformed['EnclosedPorch'] + 
                X_transformed['3SsnPorch'] + 
                X_transformed['ScreenPorch']
            )
            added_columns.append('TotalPorchSF')
        
        if all(col in X_transformed.columns for col in ['YrSold', 'YearBuilt', 'YearRemodAdd']):
            X_transformed['HouseAge'] = X_transformed['YrSold'] - X_transformed['YearBuilt']
            X_transformed['RemodelAge'] = np.where(
                X_transformed['YearRemodAdd'] == X_transformed['YearBuilt'], 
                0, 
                X_transformed['YrSold'] - X_transformed['YearRemodAdd']
            )
            X_transformed['IsNew'] = (X_transformed['YearBuilt'] == X_transformed['YrSold']).astype(int)
            added_columns += ['HouseAge', 'RemodelAge', 'IsNew']
            
        bath_cols = ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']
        if all(col in X_transformed.columns for col in bath_cols):
            X_transformed['TotalBathrooms'] = (
                X_transformed['FullBath'] + 
                0.5 * X_transformed['HalfBath'] + 
                X_transformed['BsmtFullBath'] + 
                0.5 * X_transformed['BsmtHalfBath']
            )
            added_columns.append('TotalBathrooms')
        
        binary_flags = {
            'PoolArea': 'HasPool',
            'Fireplaces': 'HasFireplace',
            'GarageArea': 'HasGarage',
            'TotalBsmtSF': 'HasBsmt'
        }
        for base_col, new_col in binary_flags.items():
            if base_col in X_transformed.columns:
                X_transformed[new_col] = (X_transformed[base_col] > 0).astype(int)
                added_columns.append(new_col)
        
        if all(col in X_transformed.columns for col in ['OverallQual', 'TotalSF']):
            X_transformed['OverallQual_TotalSF'] = X_transformed['OverallQual'] * X_transformed['TotalSF']
            added_columns.append('OverallQual_TotalSF')
        if all(col in X_transformed.columns for col in ['KitchenQual', 'GrLivArea']):
            X_transformed['KitchenQual_GrLivArea'] = X_transformed['KitchenQual'].map(
                {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0}
            ) * X_transformed['GrLivArea']
            added_columns.append('KitchenQual_GrLivArea')

        if 'OverallQual_TotalSF' in X_transformed.columns:
            X_transformed['OverallQual_TotalSF'] = X_transformed['OverallQual_TotalSF'].fillna(0)

        if 'KitchenQual_GrLivArea' in X_transformed.columns:
            X_transformed['KitchenQual_GrLivArea'] = X_transformed['KitchenQual_GrLivArea'].fillna(0)

        for col, mapping in self.ordinal_mappings.items():
            if col in X_transformed.columns and col not in self.columns_to_drop:
                X_transformed[col] = X_transformed[col].map(mapping).fillna(0).astype(int)
        
        if self.valid_onehot_cols:
            X_transformed = pd.get_dummies(
                X_transformed, 
                columns=self.valid_onehot_cols, 
                drop_first=True, 
                dummy_na=True, 
                dtype=int
            )

        for col in self.high_card_cols:
            if col in X_transformed.columns:
                freq = X_transformed[col].value_counts(normalize=True)
                X_transformed[col+'_freq'] = X_transformed[col].map(freq)
                X_transformed[col+'_freq'] = X_transformed[col+'_freq'].fillna(0) 
                X_transformed.drop(col, axis=1, inplace=True)
        
        bool_cols = X_transformed.select_dtypes(include=['bool']).columns
        X_transformed[bool_cols] = X_transformed[bool_cols].astype(int)

        skewed_cols = ['LotArea', 'GrLivArea']
        for col in skewed_cols:
            if col in X_transformed.columns and col not in self.columns_to_drop:
                X_transformed[col] = np.log1p(X_transformed[col])

        if self.feature_columns_ is None:
            self.feature_columns_ = X_transformed.columns 
        else:
            X_transformed = X_transformed.reindex(columns=self.feature_columns_, fill_value=0)

        X_transformed = X_transformed.fillna(0)
        return X_transformed

In [8]:
meaningful_na_columns = [
    "Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2",
    "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond",
    "PoolQC", "Fence", "MiscFeature"
]

categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

important_numerical_cols = ["LotFrontage", "MasVnrArea", "GarageYrBlt"]

ordinal_mappings = {
    'ExterQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'ExterCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'HeatingQC': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'KitchenQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'FireplaceQu': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageQual': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageCond': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'PoolQC': {'None': 0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtExposure': {'None': 0, 'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4},
    'BsmtFinType1': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'BsmtFinType2': {'None': 0, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6},
    'GarageFinish': {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3},
    'Fence': {'None': 0, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4},
    'LandSlope': {'Sev': 0, 'Mod': 1, 'Gtl': 2},
    'PavedDrive': {'N': 0, 'P': 1, 'Y': 2},
    'LotShape': {'Reg': 4, 'IR1': 3, 'IR2': 2, 'IR3': 1, 'None': 0},
    'Functional': {'Typ': 7, 'Min1': 6, 'Min2': 5, 'Mod': 4, 'Maj1': 3, 'Maj2': 2, 'Sev': 1, 'Sal': 0}
}

nominal_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig',
    'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
    'HouseStyle', 'RoofStyle', 'RoofMatl', 'Foundation',
    'CentralAir', 'GarageType', 'MiscFeature', 'SaleType',
    'SaleCondition', 'Utilities', 'Exterior1st', 'Exterior2nd',
    'MasVnrType', 'Heating', 'Electrical'
]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train,
    test_size=0.2, 
    random_state=0
)

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

class RMSEPrinter(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y):
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        return self.model.predict(X)
    
    def score(self, X, y):
        preds = self.predict(X)
        rmse = np.sqrt(mean_squared_error(y, preds))
        print(f"\nRMSE: {rmse:.4f}")
        return -rmse  

In [11]:
processor = HousePricePreprocessor(
            meaningful_na_columns=meaningful_na_columns,
            categorical_cols=categorical_cols,
            important_numerical_cols=important_numerical_cols,
            ordinal_mappings=ordinal_mappings,
            nominal_cols=nominal_cols,
            null_threshold=0.8,
            max_onehot=4
        )

In [12]:
pipeline = Pipeline([
        ('preprocessor', processor),
        ('scaler', StandardScaler()),
        ('model', RMSEPrinter(LinearRegression()))
    ])

In [34]:
!pip install dagshub
import dagshub
!pip install mlflow
import mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 5.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      Successfully uninstalled dacite-1.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.12.2 requires dacite>=1.8, but you have dacite 1.6.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 58.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 100.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.6 MB/s eta 0:00:00
   ━

In [35]:
dagshub.init(repo_owner='asurm22', repo_name='ML_1_asurm', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=4761f22f-c642-4d83-86a2-302fb18f9a21&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=298e15a720bd4ee2d5b79e915946668ad72ab7df2f0191ffe51d99d9174af5e4




Output()

Accessing as asurm22

Initialized MLflow to track repo "asurm22/ML_1_asurm"

Repository asurm22/ML_1_asurm initialized!

In [43]:
import mlflow
from sklearn.metrics import r2_score

mlflow.set_experiment('linear_reg')

with mlflow.start_run(run_name='run1'):    
    pipeline.fit(X_train, np.log1p(y_train))
    val_preds_log = pipeline.predict(X_val)
    val_preds = np.expm1(val_preds_log)    
    rmse = np.sqrt(mean_squared_error(np.log1p(y_val), val_preds_log))
    r2 = r2_score(y_val, val_preds) 
    
    
    mlflow.log_metrics({
        "RMSE": rmse,
        "R2": r2
    })
    
    
    mlflow.log_params({
        "null_threshold": 0.8,
        "max_onehot": 4,
        "target_transform": "log1p"
    })
    
    
    mlflow.sklearn.log_model(
        pipeline.named_steps['model'].model, 
        "model",
        input_example=X_train[:1],
        registered_model_name="HousePricePredictor"
    )
    
    
    print(f"Logged run: {mlflow.active_run().info.run_id}")
    print(f"Validation RMSE: {rmse:.4f}, R2: {r2:.4f}")


/usr/local/lib/python3.10/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
2025/04/10 16:02:39 WARNING mlflow.models.signature: Failed to infer the model sig

Logged run: c08bd4d409694c329c2d743e8aec8ffa
Validation RMSE: 0.1636, R2: 0.5520
🏃 View run run1 at: https://dagshub.com/asurm22/ML_1_asurm.mlflow/#/experiments/2/runs/c08bd4d409694c329c2d743e8aec8ffa
🧪 View experiment at: https://dagshub.com/asurm22/ML_1_asurm.mlflow/#/experiments/2


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

RFGmodel = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    min_samples_leaf=3,
    random_state=42,
    n_jobs=-1
)

RFGpipeline = Pipeline([
    ('preprocessor', processor),
    ('scaler', StandardScaler()),
    ('model', RFGmodel)
])

mlflow.set_experiment('RandomForestRegressor')

with mlflow.start_run(run_name='RFGrun1'):
    RFGpipeline.fit(X_train, np.log1p(y_train))
    
    val_preds_log = RFGpipeline.predict(X_val)
    val_preds = np.expm1(val_preds_log)

    rmse = np.sqrt(mean_squared_error(np.log1p(y_val), val_preds_log))
    r2 = r2_score(y_val, val_preds)
    
    mlflow.log_metrics({"val_rmse": rmse, "val_r2": r2})
    mlflow.sklearn.log_model(pipeline, "RFGmodel")
    
    print(f"Validation RMSE: {rmse:.4f}, R2: {r2:.4f}")

2025/04/10 15:52:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Validation RMSE: 0.1403, R2: 0.8471
🏃 View run RFGrun1 at: https://dagshub.com/asurm22/ML_1_asurm.mlflow/#/experiments/3/runs/e1c6ef84745a40f682bebf872425dcd4
🧪 View experiment at: https://dagshub.com/asurm22/ML_1_asurm.mlflow/#/experiments/3


In [31]:
log_preds = pipeline.predict(test)
safe_preds = np.expm1(np.clip(log_preds, None, 14))

submission = pd.DataFrame({
    'Id': test['Id'],
    'SalePrice': safe_preds
})
submission.to_csv('submission.csv', index=False)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


1202603.2841647768